In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import math


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [44]:
#We first read the 2 data files
df1 = pd.read_csv('pirvision_office_dataset1.csv')
df2 = pd.read_csv('pirvision_office_dataset2.csv')

In [45]:
# #We first shuffle these 2 dataframes
# df1 = df1.sample(frac = 1, random_state=1).reset_index(drop=True)
# df2 = df2.sample(frac = 1, random_state=1).reset_index(drop=True)

#We now merge these 2 dataframes
df = pd.concat([df1, df2], ignore_index=True)

#We print the shapes of all datafmrames
print(df1.shape, df2.shape, df.shape)

#Displaying the merged dataframe
# display(df.head(100))


(7651, 59) (7651, 59) (15302, 59)


In [46]:
df[df['Label'] == 3]['Temperature_F'].value_counts()


Temperature_F
0    1142
Name: count, dtype: int64

In [47]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]


class transformer(nn.Module):
    def __init__(self, input_size=1, seq_len=55, d_model=64, nhead=4, num_layers=5, num_classes=3):
        super().__init__()
        self.input_proj = nn.Linear(input_size, d_model)  # project input to d_model
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=128, dropout=0.1, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size=1)
        x = self.input_proj(x)       # (batch_size, seq_len, d_model)
        x = self.pos_encoder(x)      # (batch_size, seq_len, d_model)
        x = self.transformer_encoder(x)  # (batch_size, seq_len, d_model)
        x = x.mean(dim=1)            # mean pooling over time steps
        return self.classifier(x)    # (batch_size, num_classes)


In [48]:
print("Class distribution in df:")
print(df["Label"].value_counts())


Class distribution in df:
Label
0    12494
1     1666
3     1142
Name: count, dtype: int64


In [49]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sequence = self.X[idx][1:]
        label = self.y[idx]
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(1)  # shape: (seq_len, 1)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return sequence_tensor, label_tensor

X = df.drop(columns=["Label", "Date", "Time"]).values

label_map = {0: 0, 1: 1, 3: 2}
y_raw = df["Label"].values
y = np.array([label_map[label] for label in y_raw])

kf = KFold(n_splits=5, shuffle=True, random_state=0)

input_size = 1
model = transformer(input_size=input_size)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

# Use in CrossEntropyLoss
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = optim.Adam(model.parameters(), lr=0.001)

nb_epochs = 8
batch_size = 64  # you can adjust this based on memory

for fold, (train_i, test_i) in enumerate(kf.split(X), 1):
    print(f"\nFold {fold}")
    if fold == 2:
        break

    X_train, X_test = X[train_i], X[test_i]
    y_train, y_test = y[train_i], y[test_i]

    print("Train label distribution:", np.bincount(y_train))
    print("Test label distribution:", np.bincount(y_test))

    # --- Create DataLoader ---
    train_dataset = TimeSeriesDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(nb_epochs):
        total_loss = 0.0
        model.train()
        for sequences, labels in train_loader:
            sequences = sequences  # shape: (batch_size, seq_len, 1)
            labels = labels        # shape: (batch_size)

            output = model(sequences)  # shape: (batch_size, num_classes)
            loss = criterion(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * sequences.size(0)

        avg_loss = total_loss / len(train_dataset)
        print(f"epoch {epoch+1}, loss: {avg_loss:.4f}")

    from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

    # --- Evaluation ---
    print("testing time")
    model.eval()
    y_true = []
    y_pred = []

    correct = 0
    total = 0
    class_counts = {0: 0, 1: 0, 2: 0}
    class_correct = {0: 0, 1: 0, 2: 0}

    with torch.no_grad():
        for i in range(X_test.shape[0]):
            x = X_test[i]
            sequence = x[1:]
            label = y_test[i]
            sequence_tensor = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(2)

            output = model(sequence_tensor)
            predicted_class = torch.argmax(output, dim=1).item()

            y_true.append(label)
            y_pred.append(predicted_class)

            class_counts[label] += 1
            if predicted_class == label:
                correct += 1
                class_correct[label] += 1
            total += 1

    accuracy = correct / total
    print(f"\nfold {fold} test accuracy: {accuracy:.4f}")

    print("\nPer-class accuracy:")
    for cls in class_counts:
        total_cls = class_counts[cls]
        correct_cls = class_correct[cls]
        acc_cls = correct_cls / total_cls if total_cls > 0 else 0.0
        print(f"  Class {cls}: {correct_cls}/{total_cls} correct ({acc_cls * 100:.2f}%)")

    # --- Classification Metrics ---
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    macro_f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Macro F1-Score: {macro_f1:.4f}")



Fold 1
Train label distribution: [10041  1321   879]
Test label distribution: [2453  345  263]
epoch 1, loss: 1.0378
epoch 2, loss: 0.9625
epoch 3, loss: 0.8106
epoch 4, loss: 0.8772
epoch 5, loss: 1.0007
epoch 6, loss: 0.9628
epoch 7, loss: 0.9966
epoch 8, loss: 0.9972
testing time

fold 1 test accuracy: 0.8252

Per-class accuracy:
  Class 0: 2453/2453 correct (100.00%)
  Class 1: 0/345 correct (0.00%)
  Class 2: 73/263 correct (27.76%)

Classification Report:
              precision    recall  f1-score   support

           0     0.8210    1.0000    0.9017      2453
           1     0.0000    0.0000    0.0000       345
           2     1.0000    0.2776    0.4345       263

    accuracy                         0.8252      3061
   macro avg     0.6070    0.4259    0.4454      3061
weighted avg     0.7438    0.8252    0.7599      3061

Macro F1-Score: 0.4454

Fold 2


c:\Users\Admin\anaconda3\envs\assignmentenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\envs\assignmentenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\envs\assignmentenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)